In [1]:
#!pip install --upgrade --quiet google-genai
#!pip install --upgrade --quiet faker
#!pip install --upgrade --quiet transformers[torch] 
#!pip install --upgrade --quiet bitsandbytes
#!pip install --upgrade --quiet grandalf
#!pip install --upgrade --quiet langchain-community langchainhub langgraph
#!pip install --upgrade --quiet datasets
#!pip install --upgrade --quiet python-dotenv

import pandas as pd
import re
import os
from dotenv import load_dotenv
from typing_extensions import TypedDict
from langgraph.graph import START, StateGraph
from google import genai
import warnings as wr
wr.filterwarnings(action='ignore')

In [2]:
fichier_csv_path = 'invoice_dataset.csv'
invoice_df = pd.read_csv(fichier_csv_path)

In [10]:
load_dotenv()
client = genai.Client(api_key = os.getenv("GEMINI_API_KEY"))

def generate(prompt):
    model_name="gemini-2.0-flash"
    response = client.models.generate_content(
        model=model_name,
        contents=prompt
    )
    return response.text.rstrip()

In [11]:
class State(TypedDict):
    question: str
    invoice_ids: str
    invoice_info: str
    answer: str

# Outil pour extraire les numéros de facture
def extract_invoice_ids(state: State):
    prompt = f"""
    Vous êtes un assistant serviable conçu pour extraire les numéros de facture des questions de l'utilisateur.
    Veuillez extraire tous les numéros de facture mentionnés dans la question suivante, même s'ils sont mentionnés de manière informelle ou contiennent des erreurs de frappe.
    Les numéros de facture commencent par 'FAC-' suivi de quatre chiffres, et sont compris entre FAC-0001 et FAC-1000.
    Si une plage de factures est mentionnée  veuillez extraire tous les numéros de facture pertinents, 
    Par exemple:
        - "les 5 premières factures" doit retourner 'FAC-0001,FAC-0002,FAC-0003,FAC-0004,FAC-0005'
        - "les 5 dernières factures" doit retourner 'FAC-0996,FAC-0997,FAC-0998,FAC-0999,FAC-1000'
    Si plusieurs numéros de facture sont présents, séparez-les par des virgules.
    Si aucun numéro de facture n'est trouvé, veuillez répondre par 'AUCUN'.
    N'incluez aucune description ou contexte supplémentaire.

    Question: {state['question']}
    """
    return {"invoice_ids": generate(prompt)}


# Outil pour obtenir les informations de la facture
def get_invoice_information(state: State):
    invoice_ids_str = state["invoice_ids"]
    if invoice_ids_str.upper() == "AUCUN":
        return {"invoice_info": "Aucun numéro de facture trouvé dans la question."}
    else:
        invoice_id_matches = re.findall(r'FAC-\d{4}', invoice_ids_str, re.IGNORECASE)
        clean_keys = [id.strip().upper() for id in invoice_id_matches]
        prompt_injection = ""
        
        for key in clean_keys:
            matching_invoice = invoice_df[invoice_df["Numéro de Facture"].str.upper() == key]
            
            if not matching_invoice.empty:
                prompt_injection += f"Données de facture numéro: {key} \n"
                for column, value in matching_invoice.iloc[0].items():
                    if column == "Numéro de Facture":
                        continue
                    if pd.notna(value):
                        prompt_injection += f"  {column}: {value}\n"
            else:
                prompt_injection += f"Avertissement : Le numéro de facture '{key}' n'a pas été trouvé.\n"
        
        return {"invoice_info": prompt_injection}

# Outil pour répondre à la question en fonction des informations de la facture
def answer_based_on_info(state: State):
    question = state["question"]
    invoice_info = state["invoice_info"]
    prompt = f"""
    Vous êtes un assistant IA expert en analyse de factures. Votre rôle est de répondre aux questions posées sur les factures, en utilisant exclusivement les données fournies.

    **Question:** {question}

    **Données Fournies:**
    {invoice_info}

    Répondez à la question de manière très concise et précise, en utilisant uniquement les données fournies ci-dessus.
    N'utilisez aucune connaissance externe et ne faites pas d'hypothèses.
    Si la question nécessite des calculs, assurez-vous d'être précis et présentez les étapes importantes clairement.
    Statuts de paiement : ["Payé", "Non Payé", "En Retard", "Partiellement Payé"]  
    Modes de paiement : ["Carte de Crédit", "Virement Bancaire", "Chèque", "Espèces", "PayPal"]  
    Les champs "Date de Paiement" et "Mode de Paiement" sont renseignés uniquement si le statut est "Payé".  
    Si ces champs sont absents, cela signifie que le paiement n'est pas finalisé.  
    Ne dites jamais que les données sont insuffisantes. Indiquez simplement que le paiement n'est pas encore finalisé.
    Si une date est demandée, donnez la dans un format clair, comme le jour/mois/année.
    """
    return {'answer': generate(prompt)}

In [12]:
graph_builder = StateGraph(State).add_sequence(
    [extract_invoice_ids, get_invoice_information, answer_based_on_info]
)
graph_builder.add_edge(START, "extract_invoice_ids")
graph = graph_builder.compile()
print(graph.get_graph().draw_ascii())

def Answer(questions, show={"extract":False, "get":False, "answer":True}):
    for question in questions:
        print(f"\n--- Question: {question} ---")
        for step in graph.stream(
            {"question": question}, stream_mode="updates"
        ):
            if "extract_invoice_ids" in step and show["extract"]==True:
                print(f"--- Numéros de Facture Extraits : {step['extract_invoice_ids']['invoice_ids']} ---")
                
            if "get_invoice_information" in step and show["get"]==True:
                print("--- Informations de Facture Récupérées: ---")
                print(step['get_invoice_information']['invoice_info'])

            if "answer_based_on_info" in step and "answer" in step["answer_based_on_info"] and show["answer"]==True:
                print(f"--- Réponse: {step['answer_based_on_info']['answer']} ---")

       +-----------+         
       | __start__ |         
       +-----------+         
              *              
              *              
              *              
  +---------------------+    
  | extract_invoice_ids |    
  +---------------------+    
              *              
              *              
              *              
+-------------------------+  
| get_invoice_information |  
+-------------------------+  
              *              
              *              
              *              
  +----------------------+   
  | answer_based_on_info |   
  +----------------------+   


In [18]:
questions = [
    "Quelle est la somme des montants totaux pour les 1000 premières factures?",
]
Answer(questions, {"extract":True, "get":False, "answer":True})


--- Question: Quelle est la somme des montants totaux pour les 1000 premières factures? ---
--- Numéros de Facture Extraits : FAC-0001,FAC-0002,FAC-0003,FAC-0004,FAC-0005,FAC-0006,FAC-0007,FAC-0008,FAC-0009,FAC-0010,FAC-0011,FAC-0012,FAC-0013,FAC-0014,FAC-0015,FAC-0016,FAC-0017,FAC-0018,FAC-0019,FAC-0020,FAC-0021,FAC-0022,FAC-0023,FAC-0024,FAC-0025,FAC-0026,FAC-0027,FAC-0028,FAC-0029,FAC-0030,FAC-0031,FAC-0032,FAC-0033,FAC-0034,FAC-0035,FAC-0036,FAC-0037,FAC-0038,FAC-0039,FAC-0040,FAC-0041,FAC-0042,FAC-0043,FAC-0044,FAC-0045,FAC-0046,FAC-0047,FAC-0048,FAC-0049,FAC-0050,FAC-0051,FAC-0052,FAC-0053,FAC-0054,FAC-0055,FAC-0056,FAC-0057,FAC-0058,FAC-0059,FAC-0060,FAC-0061,FAC-0062,FAC-0063,FAC-0064,FAC-0065,FAC-0066,FAC-0067,FAC-0068,FAC-0069,FAC-0070,FAC-0071,FAC-0072,FAC-0073,FAC-0074,FAC-0075,FAC-0076,FAC-0077,FAC-0078,FAC-0079,FAC-0080,FAC-0081,FAC-0082,FAC-0083,FAC-0084,FAC-0085,FAC-0086,FAC-0087,FAC-0088,FAC-0089,FAC-0090,FAC-0091,FAC-0092,FAC-0093,FAC-0094,FAC-0095,FAC-0096,FAC-0097,

In [ ]:
questions = [
    # "Donne-moi les détails pour la facture numéro FAC-0001.",
    # "Quelle est la date de facturation pour la facture numéro FAC-0001 ?",
    # "Quelle est la date d'échéance pour la facture numéro FAC-0001 ?",
    # "Quel est le client pour la facture numéro FAC-0001 ?",
    # "Quelle est l'adresse de facturation pour la facture numéro FAC-0001 ?",
    # "Quelle est l'adresse de livraison pour la facture numéro FAC-0001 ?",
    # "Quel article a été décrit dans la facture numéro FAC-0001 ?",
    # "Quelle était la quantité de l'article dans la facture numéro FAC-0001 ?",
    # "Quel était le prix unitaire dans la facture numéro FAC-0001 ?",
    # "Quel était le montant total de l'article dans la facture numéro FAC-0001 ?",
    # "Quel est le statut du paiement pour la facture numéro FAC-0001 ?",
    # "Quelle était la date de paiement pour la facture numéro FAC-0001 ?",
    # "Quel était le mode de paiement pour la facture numéro FAC-0001 ?",
    # "Donne-moi que les détails importants pour les factures FAC-0001, FAC-0002, FAC-0003, FAC-0004.",
    # "Donne-moi les noms des clients pour les factures FAC-0001 et FAC-0002.",
    # "Quels sont les statuts de paiement pour les factures FAC-0001, FAC-0002 et FAC-0003 ?",
    # "Compare les montants totaux des factures FAC-0001 et FAC-0002. Lequel est le plus élevé ?",
    # "Les factures FAC-0001 et FAC-0002 sont-elles pour le même client ?",
    # "Quelles sont les dates d'échéance pour les factures FAC-0001 et FAC-0002 ?",
    
    # "Quelle est la somme des montants totaux pour les factures FAC-0001, FAC-0002, FAC-0003, FAC-0004 ?",
    # "Quelle est la moyenne des montants totaux pour les factures FAC-0001, FAC-0002, FAC-0003 ?",
    # "Quelle est la somme des montants totaux pour les factures FAC-0001, FAC-0005 et FAC-0010 ?",
    # "Quelle est la somme des montants totaux pour les 5 premières factures?",
    # "Quelle est la somme des montants totaux pour les 3 premières factures?",
    # "Quelle est la somme des montants totaux pour les 3 dernières factures?",
    
    # "Calculez la moyenne des prix unitaires pour les factures FAC-0002 à FAC-0005.",
    # "Quel est le montant total des factures payées par carte de crédit pour les factures FAC-0018 à FAC-0020 ?",
    # "Calculez le délai de paiement moyen (date d'échéance moins date de facturation) pour les factures FAC-0018 à FAC-0025.",
    # "Quels sont les articles qui ont été achetés en quantité supérieure à 2 dans les factures FAC-0025 à FAC-0029 ?",
    # "Pour les factures FAC-0028 et FAC-0029, quel est le client qui a le montant total le plus élevé ?",
    # "Quelle est la différence entre le montant total de la facture FAC-0035 et le montant total de la facture FAC-0036 ?",
    # "Quels sont les modes de paiement utilisés pour les factures FAC-0038 à FAC-0040 ?",
    # "Quels sont les statuts de paiement des factures FAC-0041, FAC-0042 et FAC-0043 ?",
    
    "Les factures FAC-0045 et FAC-0046 sont-elles pour le même client ?",
    "Quelles sont les factures parmi FAC-0048, FAC-0049 et FAC-0050 qui ont un statut de paiement 'En retard' ?",
    "Comparez les dates d'échéance des factures FAC-0052 et FAC-0053. Quelle facture a la date d'échéance la plus proche ?",
    "Quels sont les clients qui ont des adresses de facturation et de livraison différentes pour les factures FAC-0055 et FAC-0056 ?",
    "Quelles sont les factures parmi FAC-0065 et FAC-0066 qui ont été payées par virement bancaire ?",
    "Quel est le délai entre la date de facturation et la date de paiement pour la facture FAC-0068 ?",
]
Answer(questions, {"extract":False, "get":False, "answer":True})


--- Question: Les factures FAC-0045 et FAC-0046 sont-elles pour le même client ? ---
--- Réponse: Non, les factures FAC-0045 et FAC-0046 ne sont pas pour le même client. FAC-0045 est pour Ferrand S.A.S. et FAC-0046 est pour Blondel SA. ---

--- Question: Quelles sont les factures parmi FAC-0048, FAC-0049 et FAC-0050 qui ont un statut de paiement 'En retard' ? ---
--- Réponse: Seule la facture FAC-0049 a un statut de paiement "Non Payé". Il n'y a aucune facture avec le statut "En Retard" parmi FAC-0048, FAC-0049 et FAC-0050. ---

--- Question: Comparez les dates d'échéance des factures FAC-0052 et FAC-0053. Quelle facture a la date d'échéance la plus proche ? ---
--- Réponse: La facture FAC-0052 a une date d'échéance au 04/07/2024 et la facture FAC-0053 au 16/04/2025. La facture FAC-0052 a la date d'échéance la plus proche. ---

--- Question: Quels sont les clients qui ont des adresses de facturation et de livraison différentes pour les factures FAC-0055 et FAC-0056 ? ---
--- Réponse: 